In [11]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AACAnimalShelterCRUD import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accUser"
password = "a123"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.H1('Christal Willett Animal Rescue Training Dashboard'), # unique name identifier included
    html.Hr(),
    html.Div([ #FIXME Add code for interactive filtering options. Ex: Radio buttons, drop down, checkboxes, etc.
        dcc.Dropdown( # drop down menu to filter animal documents
            id = 'dropdown-filtering',
            options = [ # dropdown options
                {'label' : 'Water Resue', 'value' : 'water'},
                {'label' : 'Mountain & Wilderness Resue', 'value' : 'mountain/wilderness'},
                {'label' : 'Disaster Rescue & Individual Tracking', 'value' : 'disaster/tracking'},
                {'label' : 'Other program options', 'value' : 'OTHER'},
                {'label' : 'No Filter', 'value' : 'nofilter'}
            ],
            value = 'No Filter',
            multi = True
        ),
        html.Div(id='dd-output-container')
    ]),
              
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        filter_action = 'native',
        sort_action='native',
        sort_mode='native',
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action='native',
        page_current=0,
        pages_size=15,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input(value, 'value')])
def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if (value == 'water'):
        df = pd.DataFrame.from_records(shelter.readAll({'breed': {'$in' : ['Labrador Retriever Mix', 
                                                                           'Chesapeake Bay Retriever', 
                                                                           'Newfoundland']}, 
                                                     'sex_upon_outcome' : 'Intact Female'},
                                                    {'$and' : [{'age_upon_outcome_in_weeks' : {'$gte' : 26}},
                                                              {'age_upon_outcome_in_weeks' : {'$lte' : 156}}
                                                              ]}
                                                      )
                                      )
    elif (value == 'mountain/wilderness'):
        df = pd.DataFrame.from_records(shelter.readAll({'breed': {'$in' : ['German Shepherd', 
                                                                           'Alaskan Malamute', 
                                                                           'Old English Sheepdog', 
                                                                           'Siberian Husky', 'Rottweiler']}, 
                                                     'sex_upon_outcome' : 'Intact Male'},
                                                    {'$and' : [{'age_upon_outcome_in_weeks' : {'$gte' : 26}},
                                                              {'age_upon_outcome_in_weeks' : {'$lte' : 156}}
                                                              ]})
                                      )
    elif (value == 'disaster/tracking'):
        df = pd.DataFrame.from_records(shelter.readAll({'breed': {'$in' : ['Doberman Pinscher', 
                                                                           'German Shepherd', 
                                                                           'Golden Retriever', 
                                                                           'Bloodhound', 'Rottweiler']}, 
                                                     'sex_upon_outcome' : 'Intact Male'},
                                                    {'$and' : [{'age_upon_outcome_in_weeks' : {'$gte' : 20}},
                                                              {'age_upon_outcome_in_weeks' : {'$lte' : 300}}
                                                              ]})
                                      )
    elif (value == 'OTHER'):
        df = pd.DataFrame.from_records(shelter.readAll({'breed': {'$in' : []}, 
                                                     'sex_upon_outcome' : 'Intact Male'},
                                                    {'$and' : [{'age_upon_outcome_in_weeks' : {'$gt' : 500}}
                                                              ]
                                                    })
                                      )
    elif (value == 'nofilter'):
        df = pd.DataFrame.from_records(shelter.readAll({'breed': {'$in' : []}, 
                                                     'sex_upon_outcome' : ''},
                                                    {'$and' : [{'age_upon_outcome_in_weeks' : {}}
                                                              ]
                                                    })
                                      )
    
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    return [
        dcc.Graph(
            id = 'graph_id',
            figure = px.pie(dff, value = 'No Filter',
                            names = 'animal_type',
                            title = 'Available Breeds' # pie chart
                           )
        )
    ]
                               
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    viewDF = pd.DataFram.from_dict(viewData)
    dff = viewDF.loc[rows],
    return [ # geolocation chart
        dl.Map(style =  {'width' : '1000px', 'height': '500px'}, # chart properties
               center = ['location_lat', 'location_long'], zoom = 15, children = [
                   dl.TileLayer(id = 'base-layer-id'),
                   dl.Marker(position = [ # marker/label properties
                       dff.loc[0, 'location_lat'],
                       dff.loc[0, 'location_long']],
                             children = [
                                 dl.Tooltip(dff['breed']),
                                 dl.Popup([
                                     html.H1('Animal Name'),
                                     html.P(dff.iloc[0, 'name'])
                                 ])
                             ])
               ])
    ]

app

SyntaxError: invalid syntax (<ipython-input-11-c41000cb8d36>, line 42)